# Getting Started with Xanthos

In this section, we will introduce
* How to install and run xanthos
* How to use Xanthos functions

## Install Xanthos

* Download and install Python (>= 3.6).
* From a command prompt, install the latest Xanthos version from Github using
> `python -m pip install git+https://github.com/JGCRI/xanthos.git`
* (Optional) Clone Xanthos to your local folder if you would like to make modifications to `Xanthos`. From a command prompt, navigate to your preferred location, and clone `Xanthos`.
> `git clone https://github.com/JGCRI/xanthos.git`

## Download Example Data

`Xanthos` provides a function `get_package_data` to retrieve and download example data. The example data provided in this tutorial uses WATCH climate forcing data from 1971 to 2001.

To download, open a python console. Make sure to change the `data_dir` to your preferred location. This could take about 10 minutes.

In [ ]:
import xanthos

# the directory that you want to download and extract the example data to
data_dir = '<my data download location>'

# download and unzip the package data to your local machine
xanthos.get_package_data(data_dir)

## Setup the Configuration File

Setup your configuration file (.ini). Examples are located in the `example` directory that you just downloaded. Make sure to change the following variables to represent the local path to your example data: `RootDir`, `TempMinFile`, `PrecipitationFile`. Let's take a look at the example configuration file `pm_abcd_mrtm.ini`.

In [ ]:
import os

# root directory
data_dir = '<my data download location>'

# the path and file name that my example configuration (.ini) file was downloaded to
config_file = os.path.join(data_dir, 'example', 'pm_abcd_mrtm.ini')

# take a look at configuration file
f = open(config_file, 'r')
print(f.read())
    
f.close()

## Run Xanthos

To run xanthos, pass the path of the configuration file to `xanthos.run_model`.

In [ ]:
import xanthos

# run Xanthos 
xanthos.run_model(config_file)

## Use Xanthos Module

User can use any Xantho module individually by providing required input. The following example shows how to calculate accessible water using the `AccessibleWater` module. `AccessibleWater` module requires three inputs: 
1. Configuration object: use xanthos module `ConfigReader` to load
2. Loaded data object: use xanthos module `DataLoader` to load
3. Monthly runoff in each grid cell
    * Format requirement: 2D array ordered by Xanthos id for [grid_cell, month].
    * Grid cell resolution: Xanthos grid cell is at 0.5 degree xanthos grid resolution and the id is from 1 to 67420. Please find latitude and longitude for Xanthos grid cells in `example/input/referece/coordinate.csv`.
    * Unit requirement: the runoff values should be in mm/month.
    * Note: please make sure the monthly runoff has the same time period specified in the configuration file

In [ ]:
import numpy as np
from xanthos import ConfigReader
from xanthos.data_reader.data_load import DataLoader
from xanthos.accessible.accessible import AccessibleWater

# since we do not have monthly runoff, we make a ramdom example of runoff time seris
# the runoff data will be 67420 grid cells by 372 months (match 1971 - 2001 in the configuration)
runoff = np.random.random((67420, 372))

# load configuration setting using ConfigReader
config = ConfigReader(ini=config_file)

# Create a demo folder 
demo_dir = os.path.join(data_dir, 'example', 'output', 'demo')
if not os.path.exists(demo_dir):
    os.makedirs(demo_dir, exist_ok=True)

# Update the output folder path to demo folder to avoid overwriting the exisitng file
config.OutputFolder = demo_dir
config.update({})

# load reference data
data = DataLoader(config_obj=config)

# calculate accessible water (~ 15 seconds)
AccessibleWater(settings=config, ref=data, runoff=runoff)

***

# Input and Output Data

In this section, we will
* Learn the data format for Xanthos inputs and outputs
* Visualize Xanthos inputs and outputs

## Plotting Functions

In [ ]:
import xanthos

import os
from scipy import stats
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib import cm
import seaborn as sns

def name_columns(arr, start_yr=1971):
    """
    find cooresponding year month names for the array.

    :param arr:             array with format [grid by year-month]
    :param start_yr:        integer of data start year

    :return:                string vector and dictionary
    """

    cols_str = []
    cols_dict = {}
    for i in range(0, arr.shape[1], 1):
        mod = i%12
        str_yr = str(start_yr)

        if mod == 0:
            cols_str.append(str_yr)

            if str_yr in cols_dict:
                cols_dict[str_yr].append(i)
            else:
                cols_dict[str_yr] = [i]

        elif mod == 11:
            cols_str.append(str_yr)

            if str_yr in cols_dict:
                cols_dict[str_yr].append(i)
            else:
                cols_dict[str_yr] = [i]

            start_yr += 1
        else:
            cols_str.append(str_yr)

            if str_yr in cols_dict:
                cols_dict[str_yr].append(i)
            else:
                cols_dict[str_yr] = [i]

    return cols_str, cols_dict

def arr_to_df(arr, stat='mean', start_yr=1971, from_yr=False, through_yr=False):
    """
    Calculate certain stats on xanthos input array (usually climate data) and convert to certain data frame format.

    :param arr:                 array for xanthos input or similar format [grid by year-month]
    :param stat:                string for stats to apply ['min', 'max', 'mean', 'median']
    :param start_yr:            integer for data start year
    :param from_yr:             integer for start year to filter
    :param through_yr:          integer for through year to filter

    :return:                    data frame
    """

    cols_str, cols_dict = name_columns(arr, start_yr=start_yr)

    df = pd.DataFrame(data=arr, columns=cols_str)

    # select target years to process
    if from_yr:
        idx_start = min(cols_dict[str(from_yr)])
    else:
        idx_start = 0

    if through_yr:
        idx_end = max(cols_dict[str(through_yr)])
    else:
        idx_end = df.shape[1]

    df = df.iloc[:, idx_start:idx_end]

    df['key'] = 'value'

    if stat == 'mean':
        dfx = df.groupby('key').mean().T
    elif stat == 'max':
        dfx = df.groupby('key').max().T
    elif stat == 'min':
        dfx = df.groupby('key').min().T
    elif stat == 'median':
        dfx = df.groupby('key').median().T
    else:
        raise ValueError("Please enter stat of 'mean', 'median', 'min', 'max'.")

    return dfx

def format_data(sim, obs, id=229, from_yr=1971, through_yr=2010):
    """
    Combine sim and obs and format to [basin_id, basin_name, year, time, obs, sim].

    :param sim:                 data frame of xanthos output at basin scale
    :param obs:                 data frame of observation [basin, year, month, q]
    :param id:                  integer for selected basin id, or 'global' to aggregate all basins together
    :param from_yr:             integer for start year to filter data
    :param through_yr:          integer for end year to filter data

    :return:                    data frame
    """

    # melt all month_year columns to time and value for simulated output
    sim_melt = sim.melt(id_vars=['id', 'name'])

    # format simulated output
    sim_melt = sim_melt.rename(columns={'id': 'basin', 'variable': 'year'})
    sim_melt['year'] = sim_melt.year.astype(int)

    # aggregate observation to annual
    obs_annual = obs.groupby(['basin', 'year'], as_index=False).sum().drop(['month'], axis=1)

    # merge observation and simulation
    df = pd.merge(sim_melt, obs_annual, how='left', on=('basin', 'year'))
    df = df.rename(columns={'value': 'sim', 'q': 'obs', 'basin':'basin_id', 'name': 'basin_name'})

    # select basin and year range to plot
    if id == 'global':
        df_format = df[(df.year>=from_yr) & (df.year<=through_yr)].reset_index(drop=True)
        df_format = df_format.groupby(['year'], as_index=False).sum()
        df_format['basin_id'] = 'All Basins'
        df_format['basin_name'] = id
    else:
        df_format = df[df.basin_id.isin(id) & (df.year>=from_yr) & (df.year<=through_yr)].reset_index(drop=True)

    # add time
    df_format['time'] = pd.to_datetime(df_format.year, format='%Y')

    return df_format

def plot_single_line(dfx, title='', x_label='', y_label=''):
    """
    Plot a single time series.

    :param dfx:             data frame with a column of values
    :param title:           string for figure title
    :param x_label:         string for x axis label
    :param y_label:         string for y axis label
    """

    # plt.figure()
    with plt.style.context("seaborn-white"):
        fig, ax = plt.subplots(figsize=(16, 8), constrained_layout=False)
        dfx.plot(kind='line', legend=False, title=title, colormap="cubehelix", ax=ax)
        ax.set_title(title)
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        ax.set_facecolor('white')
        plt.show()

def plot_comparison(df_plot, y_label=''):
    """
    Plot comparison between observations and simulations.

    :param df_plot:         data frame with time, obs and sim
    :param y_label:         string for y axis label
    """

    # title
    title = str(df_plot['basin_id'].unique()[0]) + ' - ' + df_plot['basin_name'].unique()[0]

    # Plot
    with plt.style.context("seaborn-white"):
        fig, ax = plt.subplots(figsize=(10, 6), constrained_layout=False)
        df_plot.plot(x = 'time', y=['obs', 'sim'], kind='line', color=['black', 'dodgerblue'],
                     lw=3, legend=True, ax=ax)
        ax.set_title(title, fontsize=18, fontweight='bold')
        ax.set_xlabel('Time', fontsize=18)
        ax.set_ylabel(y_label, fontsize=18)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        ax.legend(fontsize=16)
        ax.grid()
        plt.show()
    
def plot_diagnostics(data, titlestr):
    """
    Plot diagnostics.
    
    :param data:            array from diagnostic output
    :param titlestr:        string for output scale ('Basin', 'Country', 'Region')
    """
    
    fig = plt.figure(figsize=(10,6), constrained_layout=False)
    ax = plt.gca()
    ax.loglog([0.01, 100000], [0.01, 100000], 'grey')
    ax.scatter(data[:, 0], data[:, 1], c='aquamarine', alpha=0.7, edgecolors='black', label='VIC_1971-2000')
    ax.scatter(data[:, 0], data[:, 2], c='salmon', alpha=0.7, edgecolors='black', label='WBM')
    ax.scatter(data[:, 0], data[:, 3], c='Blue', alpha=0.7, edgecolors='black', label='WBMc')
    ax.scatter(data[:, 0], data[:, 4], c='white', alpha=0.7, edgecolors='black', label='UNH/GRDC_1986-1995')
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.axis([0.01, 1e5, 0.01, 1e5])
    ax.legend(loc='lower right', bbox_to_anchor=(1, 0), fontsize=12)
    plt.title('Hydro Model Diagnostics at ' + titlestr + ' Scale', fontsize=14, fontweight='bold')
    plt.xlabel(r'Simulated Averaged Annual Runoff ($km^3$/yr)', fontsize=14)
    plt.ylabel(r'Averaged Annual Runoff ($km^3$/yr)', fontsize=14)

def plot_map(arr, coord, legend_label=None, cmap=None):
    """
    Plot spatial map.
    
    :param arr:             array of grid cells by year-month
    :param coord:           data frame for coordinates [lat, lon]
    :param legend_label:    string for legend label
    :param cmap:            string for color palette
    
    """

    cols_str, cols_dict = name_columns(tas, start_yr=1971)

    df = pd.DataFrame(data=arr, columns=cols_str).mean(axis=1).rename('value')

    df = pd.concat([df, coord], axis=1)

    fig = plt.figure(figsize=(15,6), constrained_layout=True)
    spec = fig.add_gridspec(1, 1)
    ax = fig.add_subplot(spec[0, 0])
    p = ax.scatter(df.lon, df.lat, s=3, c=df.value, cmap=cmap)
    legend_kwds = {'label':legend_label,
                   'shrink':1}

    cbar = plt.colorbar(p, **legend_kwds)
    cbar.ax.set_ylabel(legend_label, rotation=270)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")


## Read Input and Output Data

In [ ]:
# read runoff observation from input folder
obs_dir = os.path.join(data_dir, 'example', 'input', 'calibration')
obs_file = 'vic_watch_basin_km3_1971_2001_monthly.csv'
runoff_obs = pd.read_csv(os.path.join(obs_dir, obs_file))

# xanthos coordinates
coord = pd.DataFrame(data.coords[:, 1:3], columns=['lon', 'lat'])

# we can get temperature input from loaded data
# Load all input data using xanthos module `DataLoader`
tas = data.tair_load

# read example outputs
output_dir = os.path.join(data_dir, 'example', 'output', 'pm_abcd_mrtm_watch_1971_2001')

# runoff aggregated to basin scale
runoff_basin_file = 'Basin_runoff_km3peryear_pm_abcd_mrtm_watch_1971_2001.csv'

# runoff at 0.5 degree
runoff_grid_file = 'q_km3peryear_pm_abcd_mrtm_watch_1971_2001.csv'

# read runoff as data frame
runoff_basin = pd.read_csv(os.path.join(output_dir, runoff_basin_file))
runoff_grid = pd.read_csv(os.path.join(output_dir, runoff_grid_file))

## Xanthos Input Examples

Global climate data usually are stored in netCDF files. `xanthos` requires climate inputs as numpy files (e.g., `.np`). If users want to provide different climate inputs from the example data, please convert the climate data to required format as [grid by year-month]. Standard Xanthos resolution is 0.5 degree, and there are 67420 inland grid cells in total by year-month.

In [ ]:
# input climate data format requirement 2D array [67420 grid cells, year-month, ...]
print(tas.shape)
tas[1:5,]

In [ ]:
# input climate data time series
dfx = arr_to_df(arr=tas, stat='mean')
plot_single_line(dfx=dfx,
                 title='Global Mean Temperature',
                 y_label='Degree C')

In [ ]:
# spatial mean temperature across 1971 - 2001
plot_map(arr=tas,
         coord=coord,
         legend_label='Temperature (degree C)',
         cmap='RdBu')

In [ ]:
# global observed runoff format requirement [basin, year, month, q]
runoff_obs

## Xanthos Output Structure

Xanthos outputs are at 0.5 degree resolution in CSV files. Some outputs (e.g., runoff) are also aggregated to GCAM region, basin, and country scale.

In [ ]:
# runoff at xanthos grid cells (67420 cells)
print('The output is in the unit of', config.OutputUnitStr)
runoff_grid

In [ ]:
# runoff at basin scale (235 basins)
print('The output is in the unit of', config.OutputUnitStr)
runoff_basin

## Output Visualization

In [ ]:
# Compare time series of simulation and observation for an individual basin
df_plot = format_data(sim=runoff_basin,
                      obs=runoff_obs,
                      id=[217],
                      from_yr=1972,
                      through_yr=2001)
plot_comparison(df_plot=df_plot,
                y_label='Annual Runoff ($km^3$/year)')

In [ ]:
# Compare time series of simulation and observation for all basins together
df_plot = format_data(sim=runoff_basin,
                      obs=runoff_obs,
                      id='global',
                      from_yr=1972,
                      through_yr=2001)
plot_comparison(df_plot=df_plot,
                y_label='Annual Runoff ($km^3$/year)')

In [ ]:
# Diagnostics with other global data
runoff_diag = pd.read_csv(os.path.join(data_dir, 
                                       'example', 'output', 'pm_abcd_mrtm_watch_1971_2001', 
                                       'Diagnostics_Runoff_Basin_Scale_km3peryr.csv')).to_numpy()

# plot diagnostics
plot_diagnostics(data=runoff_diag[1:, 1:],
                 titlestr='Basin')

In [ ]:
# spatial mean runoff across 1971 - 2001
plot_map(arr=runoff_grid,
         coord=coord,
         legend_label='Runoff (km3/year)',
         cmap='viridis')